In [28]:
##### -*- coding: utf-8 -*-
"""
Example illustrating usage.
Other examples are contained in the unittests.
"""

from __future__ import division
from __future__ import print_function

import numpy as np

#rotate M towards target with orthogonal matrix T
M = np.array([[ 1.10061095,  0.47713676],
              [ 1.30095568, -0.16730989],
              [ 1.6787652 , -1.234039  ],
              [ 0.42456929, -1.28744732],
              [ 0.47105995,  0.85757736],
              [-0.05816789,  0.31683709],
              [-1.3511985 , -0.11610641],
              [ 1.80523345, -0.14549883]])



M_target = np.array([[ 1.15424697,  0.2724154 ],
                     [ 0.7893224 ,  0.66576866],
                     [-0.71227541,  0.55254571],
                     [-0.84737515,  0.41528169],
                     [-0.12133101, -1.28176304],
                     [-0.60248373, -0.5405648 ],
                     [ 0.45355659,  0.54495004],
                     [ 0.62044144, -1.83902599]])
#analytic method
T_analytic= target_rotation(M,M_target)
#numerical method using a gradient projection algorithm (GPA)
L,T = rotate_factors(M,'target',M_target,'orthogonal')
print(np.allclose(T,T_analytic,atol=1e-4))
#numerical method using a gradient projection algorithm (GPA) with lower level functions
#define objective function
vgQ = lambda L=None, A=None, T=None: vgQ_target(M_target,L=L,A=A,T=T)
#define starting point
T_start = T_analytic
#solve
L, phi, T, table = GPA(M, T=T_analytic, vgQ=vgQ, rotation_method='orthogonal')
#comparison
if np.allclose(T,T_analytic):
    print(True)
else:
    it_optim = vgQ(A=M, T=T)[0]
    an_optim = vgQ(A=M, T=T_analytic)[0]
    #print('Iterative algorithm optim: %f' % it_optim)
    #print('Analytic optim: %f' % an_optim)
    if it_optim<an_optim:
        print('Iterative algorithm is better.')
    else:
        print('Analytic result is better.')

#varimax rotation

M = np.array([[0.736500, -0.352968],
              [0.619721, 0.697631],
              [0.805787, -0.055111],
              [0.611298, 0.373090],
              [0.794874, -0.447917]])


"""
M = np.array([[ 0.91267378, -0.40868885],
              [ 0.70564088,  0.70856964],
              [ 0.99919505, -0.04011541],
              [ 0.82432659,  0.56611454],
              [ 0.89704398, -0.44194128]])
"""
L1,T= rotate_factors(M,'varimax',tol=0.0000001)
print(L1)
#varimax rotation using oblimin family instead of the orthomax family
L2,T= rotate_factors(M,'oblimin',1,'orthogonal')
print(np.allclose(L1,L2,atol=1e-4))


True
True
[[0.80475041 0.13926748]
 [0.10063433 0.92769384]
 [0.68856538 0.42214646]
 [0.28188311 0.65834892]
 [0.9073533  0.09573033]]
True


In [6]:
# -*- coding: utf-8 -*-
"""
Package with factor rotation algorithms.
This file contains a Python version of the gradient projection rotation
algorithms (GPA) developed by Bernaards, C.A. and Jennrich, R.I.
The code is based on the Matlab version of the code developed Bernaards, C.A.
and Jennrich, R.I. and is ported and made available with permission of the
authors.
Additionally, several analytic rotation methods are implemented.
References
----------
[1] Bernaards, C.A. and Jennrich, R.I. (2005) Gradient Projection Algorithms and Software for Arbitrary Rotation Criteria in Factor Analysis. Educational and Psychological Measurement, 65 (5), 676-696.
[2] Jennrich, R.I. (2001). A simple general procedure for orthogonal rotation. Psychometrika, 66, 289-306.
[3] Jennrich, R.I. (2002). A simple general method for oblique rotation. Psychometrika, 67, 7-19.
[4] http://www.stat.ucla.edu/research/gpa/matlab.net
[5] http://www.stat.ucla.edu/research/gpa/GPderfree.txt
"""

#__all__ = ['wrappers']

from factor_rotation._wrappers import rotate_factors

from factor_rotation._analytic_rotation import target_rotation, procrustes, promax

ModuleNotFoundError: No module named 'factor_rotation'

In [5]:
# -*- coding: utf-8 -*-

from __future__ import division
from __future__ import print_function
import numpy as np
import unittest

__all__=[]

def rotate_factors(A, method, *method_args, **algorithm_kwargs):
    r"""
    Subroutine for orthogonal and oblique rotation of the matrix :math:`A`.
    For orthogonal rotations :math:`A` is rotated to :math:`L` according to
    
    .. math::
        L =  AT,
        
    where :math:`T` is an orthogonal matrix. And, for oblique rotations
    :math:`A` is rotated to :math:`L` according to
    
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    
    Methods
    -------
    What follows is a list of available methods. Depending on the method
    additional argument are required and different algorithms
    are available. The algorithm_kwargs are additional keyword arguments
    passed to the selected algorithm (see the parameters section).
    Unless stated otherwise, only the gpa and
    gpa_der_free algorithm are available.
    
    Below,
    
    * :math:`L` is a :math:`p\times k` matrix;
    * :math:`N` is :math:`k\times k` matrix with zeros on the diagonal and ones elsewhere;
    * :math:`M` is :math:`p\times p` matrix with zeros on the diagonal and ones elsewhere;
    * :math:`C` is a :math:`p\times p` matrix with elements equal to :math:`1/p`;
    * :math:`(X,Y)=\operatorname{Tr}(X^*Y)` is the Frobenius norm;
    * :math:`\circ` is the element-wise product or Hadamard product.
    
    oblimin : orthogonal or oblique rotation that minimizes
        .. math::
            \phi(L) = \frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)N).
            
        For orthogonal rotations:
        * :math:`\gamma=0` corresponds to quartimax,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
        * :math:`\gamma=1` corresponds to varimax,
        * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
        For oblique rotations rotations:
    
        * :math:`\gamma=0` corresponds to quartimin,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimin.
        
        method_args:
        
        gamma : float
            oblimin family parameter
        rotation_method : string
            should be one of {orthogonal, oblique}
    
    orthomax : orthogonal rotation that minimizes
        .. math::
            \phi(L) = -\frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)),
            
        where :math:`0\leq\gamma\leq1`. The orthomax family is equivalent to 
        the oblimin family (when restricted to orthogonal rotations). Furthermore,
        * :math:`\gamma=0` corresponds to quartimax,
        * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
        * :math:`\gamma=1` corresponds to varimax,
        * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
        
        method_args:
        
        gamma : float (between 0 and 1)
            orthomax family parameter
    
    CF : Crawford-Ferguson family for orthogonal and oblique rotation wich minimizes:
        .. math::
            \phi(L) =\frac{1-\kappa}{4} (L\circ L,(L\circ L)N)
                      -\frac{1}{4}(L\circ L,M(L\circ L)),
                      
        where :math:`0\leq\kappa\leq1`. For orthogonal rotations the oblimin
        (and orthomax) family of rotations is equivalent to the Crawford-Ferguson family.
        To be more precise:
    
        * :math:`\kappa=0` corresponds to quartimax,
        * :math:`\kappa=\frac{1}{p}` corresponds to varimax,
        * :math:`\kappa=\frac{k-1}{p+k-2}` corresponds to parsimax,
        * :math:`\kappa=1` corresponds to factor parsimony.
        
        method_args:
        
        kappa : float (between 0 and 1)
            Crawford-Ferguson family parameter
        rotation_method : string
            should be one of {orthogonal, oblique}
    
    quartimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=0`
        
    biquartimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=\frac{1}{2}`
        
    varimax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=1`
        
    equamax : orthogonal rotation method
        minimizes the orthomax objective with :math:`\gamma=\frac{1}{p}`
        
    parsimax : orthogonal rotation method
        minimizes the Crawford-Ferguson family objective with :math:`\kappa=\frac{k-1}{p+k-2}`
        
    parsimony : orthogonal rotation method
        minimizes the Crawford-Ferguson family objective with :math:`\kappa=1`
    
    quartimin : oblique rotation method that minimizes
        minimizes the oblimin objective with :math:`\gamma=0`      
    quartimin : oblique rotation method that minimizes
        minimizes the oblimin objective with :math:`\gamma=\frac{1}{2}`   
        
    target : orthogonal or oblique rotation that rotates towards a target matrix :math:`H` by minimizing the objective
        .. math::
            \phi(L) =\frac{1}{2}\|L-H\|^2.
        
        method_args:
        
        H : numpy matrix
            target matrix
        rotation_method : string
            should be one of {orthogonal, oblique}
        For orthogonal rotations the algorithm can be set to analytic in which case
        the following keyword arguments are available:
        
        full_rank : boolean (default False)
            if set to true full rank is assumed    
    partial_target : orthogonal (default) or oblique rotation that partially rotates
        towards a target matrix :math:`H` by minimizing the objective:
        
        .. math::
            \phi(L) =\frac{1}{2}\|W\circ(L-H)\|^2.
        
        method_args:
        
        H : numpy matrix
            target matrix
        W : numpy matrix (default matrix with equal weight one for all entries)
            matrix with weights, entries can either be one or zero
    
    Parameters
    ---------
    A : numpy matrix (default None)
        non rotated factors
    method : string
        should be one of the methods listed above
    method_args : list
        additional arguments that should be provided with each method
    algorithm_kwargs : dictionary
        algorithm : string (default gpa)
            should be one of:
            
            * 'gpa': a numerical method
            * 'gpa_der_free': a derivative free numerical method
            * 'analytic' : an analytic method
        Depending on the algorithm, there are algorithm specific keyword
        arguments. For the gpa and gpa_der_free, the following
        keyword arguments are available:
        
        max_tries : integer (default 501)
            maximum number of iterations
        tol : float
            stop criterion, algorithm stops if Frobenius norm of gradient is
            smaller then tol
        For analytic, the supporeted arguments depend on the method, see above.
            
        See the lower level functions for more details.
        
    Returns
    -------
    The tuple :math:`(L,T)`
    
    Examples
    -------
    >>> A = np.random.randn(8,2)
    >>> L, T = rotate_factors(A,'varimax')
    >>> np.allclose(L,A.dot(T))
    >>> L, T = rotate_factors(A,'orthomax',0.5)
    >>> np.allclose(L,A.dot(T))
    >>> L, T = rotate_factors(A,'quartimin',0.5)
    >>> np.allclose(L,A.dot(np.linalg.inv(T.T)))
    """
    if 'algorithm' in algorithm_kwargs:
        algorithm = algorithm_kwargs['algorithm']
        algorithm_kwargs.pop('algorithm')
    else:
        algorithm = 'gpa'
    assert 'rotation_method' not in algorithm_kwargs, 'rotation_method cannot be provided as keyword argument'
    L=None
    T=None
    ff=None
    vgQ=None
    p,k = A.shape
    #set ff or vgQ to appropriate objective function, compute solution using recursion or analytically compute solution
    if method == 'orthomax':
        assert len(method_args)==1, 'Only %s family parameter should be provided' % method
        rotation_method='orthogonal'
        gamma = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: orthomax_objective(L=L,A=A,T=T,
                                                                     gamma=gamma,
                                                                     return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: orthomax_objective(L=L,A=A,T=T,
                                                                      gamma=gamma,
                                                                      return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'oblimin':
        assert len(method_args)==2, 'Both %s family parameter and rotation_method should be provided' % method
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        gamma = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: oblimin_objective(L=L,A=A,T=T,
                                                                    gamma=gamma,
                                                                    return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: oblimin_objective(L=L,A=A,T=T,
                                                                     gamma=gamma,
                                                                     rotation_method=rotation_method,
                                                                     return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'CF':
        assert len(method_args)==2, 'Both %s family parameter and rotation_method should be provided' % method
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        kappa = method_args[0]
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: CF_objective(L=L,A=A,T=T,
                                                               kappa=kappa,
                                                               rotation_method=rotation_method,
                                                               return_gradient=True)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: CF_objective(L=L,A=A,T=T,
                                                                kappa=kappa,
                                                                rotation_method=rotation_method,
                                                                return_gradient=False)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'quartimax':
        return rotate_factors(A, 'orthomax', 0, **algorithm_kwargs)
    elif method == 'biquartimax':
        return rotate_factors(A, 'orthomax', 0.5, **algorithm_kwargs)
    elif method == 'varimax':
        return rotate_factors(A, 'orthomax', 1, **algorithm_kwargs)
    elif method == 'equamax':
        return rotate_factors(A, 'orthomax', 1/p, **algorithm_kwargs)
    elif method == 'parsimax':
        return rotate_factors(A, 'CF', (k-1)/(p+k-2), 'orthogonal', **algorithm_kwargs)    
    elif method == 'parsimony':
        return rotate_factors(A, 'CF', 1, 'orthogonal', **algorithm_kwargs)    
    elif method == 'quartimin':
        return rotate_factors(A, 'oblimin', 0, 'oblique', **algorithm_kwargs)
    elif method == 'biquartimin':
        return rotate_factors(A, 'oblimin', 0.5, 'oblique', **algorithm_kwargs)
    elif method == 'target':
        assert len(method_args)==2, 'only the rotation target and orthogonal/oblique should be provide for %s rotation' % method
        H=method_args[0]
        rotation_method=method_args[1]
        assert rotation_method in ['orthogonal','oblique'], 'rotation_method should be one of {orthogonal, oblique}'
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: vgQ_target(H,L=L,A=A,T=T,rotation_method=rotation_method)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: ff_target(H,L=L,A=A,T=T,rotation_method=rotation_method)
        elif algorithm =='analytic':
            assert rotation_method == 'orthogonal', 'For analytic %s rotation only orthogonal rotation is supported'
            T= ar.target_rotation(A,H,**algorithm_kwargs)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    elif method == 'partial_target':
        assert len(method_args)==2, '2 additional arguments are expected for %s rotation' % method
        H=method_args[0]
        W=method_args[1]
        rotation_method='orthogonal'
        if algorithm =='gpa':
            vgQ=lambda L=None, A=None, T=None: vgQ_partial_target(H,W=W,L=L,A=A,T=T)
        elif algorithm =='gpa_der_free':
            ff = lambda L=None, A=None, T=None: ff_partial_target(H,W=W,L=L,A=A,T=T)
        else:
            raise ValueError('Algorithm %s is not possible for %s rotation' % (algorithm, method))
    else:
        raise ValueError('Invalid method')
    #compute L and T if not already done
    if T is None:
        L, phi, T, table = GPA(A, vgQ=vgQ, ff=ff, rotation_method=rotation_method, **algorithm_kwargs)
    if L is None:
        assert T is not None, 'Cannot compute L without T'
        L=rotateA(A,T,rotation_method=rotation_method)
    #return
    return L, T



In [4]:
# -*- coding: utf-8 -*-
"""
This file contains a Python version of the gradient projection rotation
algorithms (GPA) developed by Bernaards, C.A. and Jennrich, R.I.
The code is based on code developed Bernaards, C.A. and Jennrich, R.I.
and is ported and made available with permission of the authors.
References
----------
[1] Bernaards, C.A. and Jennrich, R.I. (2005) Gradient Projection Algorithms and Software for Arbitrary Rotation Criteria in Factor Analysis. Educational and Psychological Measurement, 65 (5), 676-696.
[2] Jennrich, R.I. (2001). A simple general procedure for orthogonal rotation. Psychometrika, 66, 289-306.
[3] Jennrich, R.I. (2002). A simple general method for oblique rotation. Psychometrika, 67, 7-19.
[4] http://www.stat.ucla.edu/research/gpa/matlab.net
[5] http://www.stat.ucla.edu/research/gpa/GPderfree.txt
"""

from __future__ import division
import numpy as np
import unittest

def GPA(A, ff=None, vgQ=None, T=None, max_tries=501,
       rotation_method = 'orthogonal', tol=1e-5):
    """
    The gradient projection algorithm (GPA) minimizes a target function
    :math:`\phi(L)`, where :math:`L` is a matrix with rotated factors.
    For orthogonal rotation methods :math:`L=AT`, where :math:`T` is an
    orthogonal matrix. For oblique rotation matrices :math:`L=A(T^*)^{-1}`,
    where :math:`T` is a normal matrix, i.e., :math:`TT^*=T^*T`. Oblique
    rotations relax the orthogonality constraint in order to gain simplicity
    in the interpretation.
    Parameters
    ----------
    A : numpy matrix
        non rotated factors
    T : numpy matrix (default identity matrix)
        initial guess of rotation matrix
    ff : function (defualt None)
        criterion :math:`\phi` to optimize. Should have A, T, L as keyword arguments
        and mapping to a float. Only used (and required) if vgQ is not provided.
    vgQ : function (defualt None)
        criterion :math:`\phi` to optimize and its derivative. Should have  A, T, L as
        keyword arguments and mapping to a tuple containing a
        float and vector. Can be omitted if ff is provided.
    max_tries : integer (default 501)
        maximum number of iterations
    rotation_method : string
        should be one of {orthogonal, oblique}
    tol : float
        stop criterion, algorithm stops if Frobenius norm of gradient is smaller
        then tol
    """
    #pre processing
    if rotation_method not in ['orthogonal', 'oblique']:
        raise ValueError('rotation_method should be one of {orthogonal, oblique}')
    if vgQ is None:
        if ff is None:
            raise ValueError('ff should be provided if vgQ is not')
        derivative_free=True
        Gff = lambda x: Gf(x, lambda y: ff(T=y,A=A,L=None))
    else:
        derivative_free=False
    if T is None:
        T=np.eye(A.shape[1])
    #pre processing for iteration
    al=1
    table=[]
    #pre processing for iteration: initialize f and G
    if derivative_free:
        f=ff(T=T,A=A,L=None)
        G=Gff(T)
    elif rotation_method == 'orthogonal': # and not derivative_free
        L= A.dot(T)
        f,Gq = vgQ(L=L)
        G=(A.T).dot(Gq)
    else: #i.e. rotation_method == 'oblique' and not derivative_free
        Ti=np.linalg.inv(T)
        L= A.dot(Ti.T)
        f,Gq = vgQ(L=L)
        G=-((L.T).dot(Gq).dot(Ti)).T
    #iteration
    for i_try in range(0,max_tries):
        #determine Gp
        if rotation_method == 'orthogonal':
            M=(T.T).dot(G)
            S=(M+M.T)/2
            Gp=G-T.dot(S)
        else: #i.e. if rotation_method == 'oblique':
            Gp=G-T.dot(np.diag(np.sum(T*G,axis=0)))
        s=np.linalg.norm(Gp,'fro');
        table.append([i_try, f, np.log10(s), al])
        #if we are close stop
        if s < tol: break
        #update T
        al=2*al
        for i in range(11):
            #determine Tt
            X=T-al*Gp
            if rotation_method == 'orthogonal':
                U,D,V=np.linalg.svd(X,full_matrices=False)
                Tt=U.dot(V)
            else: #i.e. if rotation_method == 'oblique':
                v=1/np.sqrt(np.sum(X**2,axis=0))
                Tt=X.dot(np.diag(v))
            #calculate objective using Tt
            if derivative_free:
                ft=ff(T=Tt,A=A,L=None)
            elif rotation_method == 'orthogonal': # and not derivative_free
                L=A.dot(Tt)
                ft,Gq=vgQ(L=L);
            else: #i.e. rotation_method == 'oblique' and not derivative_free
                Ti=np.linalg.inv(Tt)
                L= A.dot(Ti.T)
                ft,Gq = vgQ(L=L)
            #if sufficient improvement in objective -> use this T
            if ft<f-.5*s**2*al: break
            al=al/2
        #post processing for next iteration
        T=Tt
        f=ft
        if derivative_free:
            G=Gff(T)
        elif rotation_method == 'orthogonal': # and not derivative_free
            G=(A.T).dot(Gq)
        else: #i.e. rotation_method == 'oblique' and not derivative_free
            G=-((L.T).dot(Gq).dot(Ti)).T
    #post processing
    Th=T
    Lh = rotateA(A,T,rotation_method=rotation_method)
    Phi = (T.T).dot(T)
    return Lh, Phi, Th, table

def Gf(T, ff):
    """
    Subroutine for the gradient of f using numerical derivatives.
    """
    k=T.shape[0]
    ep = 1e-4
    G=np.zeros((k,k))
    for r in range(k):
        for s in range(k):
            dT=np.zeros((k,k))
            dT[r,s]=ep
            G[r,s]=(ff(T+dT)-ff(T-dT))/(2*ep);
    return G

def rotateA(A, T, rotation_method='orthogonal'):
    r"""
    For orthogonal rotation methods :math:`L=AT`, where :math:`T` is an
    orthogonal matrix. For oblique rotation matrices :math:`L=A(T^*)^{-1}`,
    where :math:`T` is a normal matrix, i.e., :math:`TT^*=T^*T`. Oblique
    rotations relax the orthogonality constraint in order to gain simplicity
    in the interpretation.
    """
    if rotation_method == 'orthogonal':
        L=A.dot(T)
    elif rotation_method == 'oblique':
        L=A.dot(np.linalg.inv(T.T))
    else: #i.e. if rotation_method == 'oblique':
        raise ValueError('rotation_method should be one of {orthogonal, oblique}')
    return L

def oblimin_objective(L=None, A=None, T=None, gamma=0,
                      rotation_method='orthogonal',
                      return_gradient=True):
    r"""
    Objective function for the oblimin family for orthogonal or
    oblique rotation wich minimizes:
    .. math::
        \phi(L) = \frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)N),
    where :math:`L` is a :math:`p\times k` matrix, :math:`N` is :math:`k\times k`
    matrix with zeros on the diagonal and ones elsewhere, :math:`C` is a
    :math:`p\times p` matrix with elements equal to :math:`1/p`,
    :math:`(X,Y)=\operatorname{Tr}(X^*Y)` is the Frobenius norm and :math:`\circ`
    is the element-wise product or Hadamard product.
    The gradient is given by
    .. math::
        L\circ\left[(I-\gamma C) (L \circ L)N\right].
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix. For oblique rotations :math:`L` satisfies
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    The oblimin family is parametrized by the parameter :math:`\gamma`. For orthogonal
    rotations:
    * :math:`\gamma=0` corresponds to quartimax,
    * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
    * :math:`\gamma=1` corresponds to varimax,
    * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
    For oblique rotations rotations:
    * :math:`\gamma=0` corresponds to quartimin,
    * :math:`\gamma=\frac{1}{2}` corresponds to biquartimin.
    Parametes
    ---------
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    gamma : float (default 0)
        a parameter
    rotation_method : string
        should be one of {orthogonal, oblique}
    return_gradient : boolean (default True)
        toggles return of gradient
    """
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method=rotation_method)
    p,k = L.shape
    L2=L**2
    N=np.ones((k,k))-np.eye(k)
    if np.isclose(gamma,0):
        X=L2.dot(N)
    else:
        C=np.ones((p,p))/p
        X=(np.eye(p)-gamma*C).dot(L2).dot(N)
    phi=np.sum(L2*X)/4
    if return_gradient:
        Gphi=L*X
        return phi, Gphi
    else:
        return phi

def orthomax_objective(L=None, A=None, T=None, gamma=0, return_gradient=True):
    r"""
    Objective function for the orthomax family for orthogonal
    rotation wich minimizes the following objective:
    .. math::
        \phi(L) = -\frac{1}{4}(L\circ L,(I-\gamma C)(L\circ L)),
    where :math:`0\leq\gamma\leq1`, :math:`L` is a :math:`p\times k` matrix,
    :math:`C` is a  :math:`p\times p` matrix with elements equal to :math:`1/p`,
    :math:`(X,Y)=\operatorname{Tr}(X^*Y)` is the Frobenius norm and :math:`\circ`
    is the element-wise product or Hadamard product.
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix.
    The orthomax family is parametrized by the parameter :math:`\gamma`:
    * :math:`\gamma=0` corresponds to quartimax,
    * :math:`\gamma=\frac{1}{2}` corresponds to biquartimax,
    * :math:`\gamma=1` corresponds to varimax,
    * :math:`\gamma=\frac{1}{p}` corresponds to equamax.
    Parametes
    ---------
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    gamma : float (default 0)
        a parameter
    return_gradient : boolean (default True)
        toggles return of gradient
    """
    assert 0<=gamma<=1, "Gamma should be between 0 and 1"
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method='orthogonal')
    p,k = L.shape
    L2=L**2
    if np.isclose(gamma,0):
        X=L2
    else:
        C=np.ones((p,p))/p
        X=(np.eye(p)-gamma*C).dot(L2)
    phi=-np.sum(L2*X)/4
    if return_gradient:
        Gphi=-L*X
        return phi, Gphi
    else:
        return phi

def CF_objective(L=None, A=None, T=None, kappa=0,
                 rotation_method='orthogonal',
                 return_gradient=True):
    r"""
    Objective function for the Crawford-Ferguson family for orthogonal
    and oblique rotation wich minimizes the following objective:
    .. math::
        \phi(L) =\frac{1-\kappa}{4} (L\circ L,(L\circ L)N)
                  -\frac{1}{4}(L\circ L,M(L\circ L)),
    where :math:`0\leq\kappa\leq1`, :math:`L` is a :math:`p\times k` matrix,
    :math:`N` is :math:`k\times k` matrix with zeros on the diagonal and ones elsewhere,
    :math:`M` is :math:`p\times p` matrix with zeros on the diagonal and ones elsewhere
    :math:`(X,Y)=\operatorname{Tr}(X^*Y)` is the Frobenius norm and :math:`\circ`
    is the element-wise product or Hadamard product.
    The gradient is given by
    .. math::
       d\phi(L) = (1-\kappa) L\circ\left[(L\circ L)N\right]
                   -\kappa L\circ \left[M(L\circ L)\right].
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix. For oblique rotations :math:`L` satisfies
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    For orthogonal rotations the oblimin (and orthomax) family of rotations is
    equivalent to the Crawford-Ferguson family. To be more precise:
    * :math:`\kappa=0` corresponds to quartimax,
    * :math:`\kappa=\frac{1}{p}` corresponds to variamx,
    * :math:`\kappa=\frac{k-1}{p+k-2}` corresponds to parsimax,
    * :math:`\kappa=1` corresponds to factor parsimony.
    Parametes
    ---------
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    gamma : float (default 0)
        a parameter
    rotation_method : string
        should be one of {orthogonal, oblique}
    return_gradient : boolean (default True)
        toggles return of gradient
    """
    assert 0<=kappa<=1, "Kappa should be between 0 and 1"
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method=rotation_method)
    p,k = L.shape
    L2=L**2
    X=None
    if not np.isclose(kappa,1):
        N=np.ones((k,k))-np.eye(k)
        X=(1-kappa)*L2.dot(N)
    if not np.isclose(kappa,0):
        M=np.ones((p,p))-np.eye(p)
        if X is None:
            X=kappa*M.dot(L2)
        else:
            X+=kappa*M.dot(L2)
    phi=np.sum(L2*X)/4
    if return_gradient:
        Gphi=L*X
        return phi, Gphi
    else:
        return phi

def vgQ_target(H, L=None, A=None, T=None, rotation_method='orthogonal'):
    r"""
    Subroutine for the value of vgQ using orthogonal or oblique rotation towards a target matrix,
    i.e., we minimize:
    .. math::
        \phi(L) =\frac{1}{2}\|L-H\|^2
    and the gradient is given by
    .. math::
        d\phi(L)=L-H.
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix. For oblique rotations :math:`L` satisfies
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    Parametes
    ---------
    H : numpy matrix
        target matrix
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    rotation_method : string
        should be one of {orthogonal, oblique}
    """
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method=rotation_method)
    q=np.linalg.norm(L-H,'fro')**2
    Gq=2*(L-H);
    return q, Gq

def ff_target(H, L=None, A=None, T=None, rotation_method='orthogonal'):
    r"""
    Subroutine for the value of f using (orthogonal or oblique) rotation towards a target matrix,
    i.e., we minimize:
    .. math::
        \phi(L) =\frac{1}{2}\|L-H\|^2.
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be
    provided. For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix. For oblique rotations :math:`L` satisfies
    .. math::
        L =  A(T^*)^{-1},
    where :math:`T` is a normal matrix.
    Parametes
    ---------
    H : numpy matrix
        target matrix
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    rotation_method : string
        should be one of {orthogonal, oblique}
    """
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method=rotation_method)
    return np.linalg.norm(L-H,'fro')**2

def vgQ_partial_target(H, W=None, L=None, A=None, T=None):
    r"""
    Subroutine for the value of vgQ using orthogonal rotation towards a partial
    target matrix, i.e., we minimize:
    .. math::
        \phi(L) =\frac{1}{2}\|W\circ(L-H)\|^2,
    where :math:`\circ` is the element-wise product or Hadamard product and :math:`W`
    is a matrix whose entries can only be one or zero. The gradient is given by
    .. math::
        d\phi(L)=W\circ(L-H).
    Either :math:`L` should be provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix.
    Parametes
    ---------
    H : numpy matrix
        target matrix
    W : numpy matrix (default matrix with equal weight one for all entries)
        matrix with weights, entries can either be one or zero
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    """
    if W is None:
        return vgQ_target(H, L=L, A=A, T=T)
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method='orthogonal')
    q=np.linalg.norm(W*(L-H),'fro')**2
    Gq=2*W*(L-H)
    return q, Gq

def ff_partial_target(H, W=None, L=None, A=None, T=None):
    r"""
    Subroutine for the value of vgQ using orthogonal rotation towards a partial
    target matrix, i.e., we minimize:
    .. math::
        \phi(L) =\frac{1}{2}\|W\circ(L-H)\|^2,
    where :math:`\circ` is the element-wise product or Hadamard product and :math:`W`
    is a matrix whose entries can only be one or zero. Either :math:`L` should be
    provided or :math:`A` and :math:`T` should be provided.
    For orthogonal rotations :math:`L` satisfies
    .. math::
        L =  AT,
    where :math:`T` is an orthogonal matrix.
    Parametes
    ---------
    H : numpy matrix
        target matrix
    W : numpy matrix (default matrix with equal weight one for all entries)
        matrix with weights, entries can either be one or zero
    L : numpy matrix (default None)
        rotated factors, i.e., :math:`L=A(T^*)^{-1}=AT`
    A : numpy matrix (default None)
        non rotated factors
    T : numpy matrix (default None)
        rotation matrix
    """
    if W is None:
        return ff_target(H, L=L, A=A, T=T)
    if L is None:
        assert(A is not None and T is not None)
        L=rotateA(A,T,rotation_method='orthogonal')
    q=np.linalg.norm(W*(L-H),'fro')**2
    return q






In [3]:
# -*- coding: utf-8 -*-
"""
This file contains analytic implementations of rotation methods.
"""

from __future__ import division
from __future__ import print_function

import numpy as np
import scipy as sp
import scipy.linalg

def target_rotation(A, H, full_rank = False):
    r"""
    Analytically performs orthogonal rotations towards a target matrix,
    i.e., we minimize:
        
    .. math::
        \phi(L) =\frac{1}{2}\|AT-H\|^2.
        
    where :math:`T` is an orthogonal matrix. This problem is also known as
    an orthogonal Procrustes problem.
        
    Under the assumption that :math:`A^*H` has full rank, the analytical
    solution :math:`T` is given by:
    
    .. math::
        T = (A^*HH^*A)^{-\frac{1}{2}}A^*H,
		
    see Green (1952). In other cases the solution is given by :math:`T = UV`,
    where :math:`U` and :math:`V` result from the singular value decomposition
    of :math:`A^*H`:
        
    .. math::
        A^*H = U\Sigma V,
    
    see Schonemann (1966).
    
    Parametes
    ---------
    A : numpy matrix (default None)
        non rotated factors
    H : numpy matrix
        target matrix
    full_rank : boolean (default FAlse)
        if set to true full rank is assumed
        
    Returns
    -------
    The matrix :math:`T`.
    
    References
    ----------
    [1] Green (1952, Psychometrika) - The orthogonal approximation of an
    oblique structure in factor analysis
	
    [2] Schonemann (1966) - A generalized solution of the orthogonal
    procrustes problem
	
    [3] Gower, Dijksterhuis (2004) - Procustes problems
    """
    ATH=A.T.dot(H)
    if full_rank or np.linalg.matrix_rank(ATH)==A.shape[1]:
        T = sp.linalg.fractional_matrix_power(ATH.dot(ATH.T),-1/2).dot(ATH)
    else:
        U,D,V=np.linalg.svd(ATH,full_matrices=False)
        T=U.dot(V)
    return T

def procrustes(A, H):
    r"""
    Analytically solves the following Procrustes problem:
        
    .. math::
        \phi(L) =\frac{1}{2}\|AT-H\|^2.
        
    (With no further conditions on :math:`H`)
    Under the assumption that :math:`A^*H` has full rank, the analytical
    solution :math:`T` is given by:
    
    .. math::
        T = (A^*HH^*A)^{-\frac{1}{2}}A^*H,
		
    see Navarra, Simoncini (2010).
    
    Parametes
    ---------
    A : numpy matrix
        non rotated factors
    H : numpy matrix
        target matrix
    full_rank : boolean (default False)
        if set to true full rank is assumed
        
    Returns
    -------
    The matrix :math:`T`.
    
    References
    ----------
    [1] Navarra, Simoncini (2010) - A guide to emprirical orthogonal functions
    for climate data analysis
    """
    return np.linalg.inv(A.T.dot(A)).dot(A.T).dot(H);

def promax(A,k=2):
    r"""
    Performs promax rotation of the matrix :math:`A`.
    
    This method was not very clear to me from the literature, this implementation
    is as I understand it should work.
    
    Promax rotation is performed in the following steps:
    
    * Deterine varimax rotated patterns :math:`V`.
    
    * Construct a rotation target matrix :math:`|V_{ij}|^k/V_{ij}
    
    * Perform procrustes rotation towards the target to obtain T
    
    * Determine the patterns
    
    First, varimax rotation a target matrix :math:`H` is determined with orthogonal varimax rotation.
    Then, oblique target rotation is performed towards the target.
    
    Parameters
    ---------
    A : numpy matrix
        non rotated factors
    k : float
        parameter, should be positive
    
    References
    ----------
    [1] Browne (2001) - An overview of analytic rotation in exploratory factor analysis
    
    [2] Navarra, Simoncini (2010) - A guide to emprirical orthogonal functions
    for climate data analysis
    """
    assert k>0
    #define rotation target using varimax rotation:
    from _wrappers import rotate_factors
    V, T = rotate_factors(A,'varimax')
    H = np.abs(V)**k/V
    #solve procrustes problem
    S=procrustes(A,H) #np.linalg.inv(A.T.dot(A)).dot(A.T).dot(H);
    #normalize
    d=np.sqrt(np.diag(np.linalg.inv(S.T.dot(S))));
    D=np.diag(d)
    T=np.linalg.inv(S.dot(D)).T
    #return
    return A.dot(T), T

